In [1]:
from pyspark.sql import SparkSession
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('vizualizações').getOrCreate()

23/11/11 19:40:11 WARN Utils: Your hostname, rodrigo-Nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp8s0)
23/11/11 19:40:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/11 19:40:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df_parquet = spark.read.parquet('/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_cities_skylines/total_video_publicado_semana/total_video_publicado_semana.parquet')
df_parquet.createOrReplaceTempView('TOTAL_PUBLICACOES')

In [5]:
spark.sql('SELECT * FROM TOTAL_PUBLICACOES WHERE NM_CANAL LIKE "C%"').show(truncate=False)

+----------------+----------------------------+------------+---------------+------------------------+
|SEMANA_TRADUZIDA|NM_CANAL                    |TOTAL_VIDEOS|DATA_PUBLICACAO|ID_CANAL                |
+----------------+----------------------------+------------+---------------+------------------------+
|Segunda-feira   |CANAL HAND - GAME WITH MUSIC|1           |2023-10-16     |UC7v51W2NoLFawizgcNuVBOA|
|Segunda-feira   |CANAL HAND - GAME WITH MUSIC|1           |2023-10-23     |UC7v51W2NoLFawizgcNuVBOA|
|Quarta-feira    |CANAL HAND - GAME WITH MUSIC|1           |2023-10-18     |UC7v51W2NoLFawizgcNuVBOA|
|Terça-feira     |CANAL HAND - GAME WITH MUSIC|1           |2023-10-17     |UC7v51W2NoLFawizgcNuVBOA|
|Sexta-feira     |CANAL HAND - GAME WITH MUSIC|1           |2023-10-20     |UC7v51W2NoLFawizgcNuVBOA|
|Sábado          |CANAL HAND - GAME WITH MUSIC|1           |2023-10-21     |UC7v51W2NoLFawizgcNuVBOA|
|Domingo         |CANAL HAND - GAME WITH MUSIC|1           |2023-10-22     |UC7v51

In [6]:
spark.sql('SELECT DATA_PUBLICACAO, sum(TOTAL_VIDEOS) FROM TOTAL_PUBLICACOES GROUP BY  DATA_PUBLICACAO').show()

+---------------+-----------------+
|DATA_PUBLICACAO|sum(TOTAL_VIDEOS)|
+---------------+-----------------+
|     2023-10-25|               23|
|     2023-10-16|               16|
|     2023-10-15|               10|
|     2023-10-18|               14|
|     2023-10-23|               24|
|     2023-10-21|               21|
|     2023-10-20|               13|
|     2023-10-19|               17|
|     2023-10-22|               15|
|     2023-10-17|               16|
|     2023-10-26|               22|
|     2023-10-24|               23|
|     2023-10-27|               21|
+---------------+-----------------+



In [7]:
spark.sql('SELECT COUNT(*) FROM TOTAL_PUBLICACOES where DATA_PUBLICACAO = "2023-10-25"  ' ).show(26)

+--------+
|count(1)|
+--------+
|      23|
+--------+



# Relação Comentários X Visualização (proporção) e Relação Visualizações X Likes (proporção)

In [8]:
assunto = 'assunto_cities_skylines'

In [9]:
df_visualizacoes_semana = spark.read.parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/{assunto}/total_visualizacoes_por_semana')
df_visualizacoes_semana.createOrReplaceTempView('METRICAS_VIDEO')

In [10]:
spark.sql(' SELECT *'
          ' FROM METRICAS_VIDEO ').show(truncate=False)

+---------------+-----------+---------------------------------------------------------------------------------------------+---------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+----------+-------------------+-----------------+-----------+-----------------------+---------------------+---------------+-------------+------------------------+
|NM_CANAL       |ID_VIDEO   |TITULO_VIDEO                                                                                 |TOTAL_CARACTERE_VIDEO|TAGS                

In [11]:
spark.sql(' SELECT ID_CANAL, NM_CANAL, count(*)'
          ' FROM METRICAS_VIDEO '
          ' GROUP BY ID_CANAL, NM_CANAL ORDER BY 3 DESC').show(truncate=False)

+------------------------+----------------------------+--------+
|ID_CANAL                |NM_CANAL                    |count(1)|
+------------------------+----------------------------+--------+
|UCrOH1V-FyMunBIMrKL0y0xQ|ChratosGameplay             |114     |
|UC7v51W2NoLFawizgcNuVBOA|CANAL HAND - GAME WITH MUSIC|83      |
|UCkXA2EoaXRFW18vrAE01BpQ|Betospa Gaming              |67      |
|UC1mk6EtfMjxR4eEZ7C43zTQ|Irmãos Cities               |67      |
|UCfY4ggUDPeecGSCNePyyVSA|MetalBear                   |58      |
|UCcn60fs5EMW8A_k9R-e4gEg|Cabrunco Games              |51      |
|UCOYJm7BsLV8VCkW2DC1MI5w|Lucas gameplays brasil      |51      |
|UCayiH2Y5JBEOSubqoQhufDA|Lives do Chratos            |48      |
|UCDGo_s0I-2JpO3o3xlkMQVA|Simularies                  |38      |
|UCrUcdH_bCfX77xpXHyq2fJw|Canal do Void               |37      |
|UCmg3_taw1BrGuo1ITlQe2hA|O Construtor                |36      |
|UCTrNnXivZU_1-aJqn1uNANg|DILSÃO                      |35      |
|UCe9jrI0YQ5SM6h5QRZ9FZlA

In [22]:
df_result = spark.sql(' SELECT ID_VIDEO, TOTAL_CARACTERE_VIDEO, TITULO_VIDEO,NM_CANAL, TOTAL_COMENTARIOS, TOTAL_LIKES , TOTAL_VISUALIZACOES '
          ' FROM METRICAS_VIDEO '
        #   ' WHERE data_extracao = "2023-10-27" ' 
          ' where ID_CANAL in ("UCrOH1V-FyMunBIMrKL0y0xQ") ').toPandas()
fig = px.scatter(df_result, x='TOTAL_CARACTERE_VIDEO', y='TOTAL_VISUALIZACOES')
df_result.sort_values(by='TOTAL_VISUALIZACOES', ascending=False).head()


,ID_VIDEO,TOTAL_CARACTERE_VIDEO,TITULO_VIDEO,NM_CANAL,TOTAL_COMENTARIOS,TOTAL_LIKES,TOTAL_VISUALIZACOES
13,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,418,13988,120812
31,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,416,13784,117375
47,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,414,13460,111842
62,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,408,13033,104409
74,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,403,12605,97039


In [23]:
fig.show()

- Comparação Likes X Visualizações e Cometários X Visualizações

In [24]:
fig = make_subplots(rows=1, cols=3, subplot_titles=['Likes X Visualizações', 'Cometários X Visualizações', 'TOTAL_CARACTERE_VIDEO X VISUALIZAÇÔES'])
sc1 = px.scatter(df_result, x='TOTAL_LIKES', y='TOTAL_VISUALIZACOES',)
fig.add_trace(sc1['data'][0], row=1, col=1)
fig.update_xaxes(title_text='Likes', row=1, col=1)
fig.update_yaxes(title_text='Visualizações', row=1, col=1)

sc2 = px.scatter(df_result, x='TOTAL_COMENTARIOS', y='TOTAL_VISUALIZACOES')
fig.add_trace(sc2['data'][0], row=1, col=2)
fig.update_xaxes(title_text='Comentários', row=1, col=2)
fig.update_yaxes(title_text='Visualizações', row=1, col=2)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)

sc3 = px.scatter(df_result, x='TOTAL_CARACTERE_VIDEO', y='TOTAL_VISUALIZACOES')
fig.add_trace(sc3['data'][0], row=1, col=3)
fig.update_xaxes(title_text='TOTAL_CARACTERE_VIDEO', row=1, col=3)
fig.update_yaxes(title_text='Visualizações', row=1, col=3)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)



fig.show()

# Dia em que os vídeos são Públicados com alta frequência

In [25]:
df_dias_publicado = spark.read.parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/{assunto}/total_video_publicado_semana')
df_dias_publicado.createOrReplaceTempView('PUBLICACAO_VIDEO')

In [26]:
spark.sql('select * from PUBLICACAO_VIDEO WHERE ID_CANAL IN ("UCrOH1V-FyMunBIMrKL0y0xQ")').show()

+----------------+---------------+------------+---------------+--------------------+
|SEMANA_TRADUZIDA|       NM_CANAL|TOTAL_VIDEOS|DATA_PUBLICACAO|            ID_CANAL|
+----------------+---------------+------------+---------------+--------------------+
|   Segunda-feira|ChratosGameplay|           1|     2023-10-23|UCrOH1V-FyMunBIMr...|
|   Segunda-feira|ChratosGameplay|           1|     2023-10-16|UCrOH1V-FyMunBIMr...|
|     Terça-feira|ChratosGameplay|           1|     2023-10-24|UCrOH1V-FyMunBIMr...|
|    Quinta-feira|ChratosGameplay|           1|     2023-10-26|UCrOH1V-FyMunBIMr...|
|    Quinta-feira|ChratosGameplay|           1|     2023-10-19|UCrOH1V-FyMunBIMr...|
|    Quarta-feira|ChratosGameplay|           1|     2023-10-18|UCrOH1V-FyMunBIMr...|
|     Terça-feira|ChratosGameplay|           1|     2023-10-17|UCrOH1V-FyMunBIMr...|
|    Quarta-feira|ChratosGameplay|           1|     2023-10-25|UCrOH1V-FyMunBIMr...|
|     Sexta-feira|ChratosGameplay|           1|     2023-10-20|UC

In [27]:
def indice_semana(dia: str) -> int: 
    dias_semana = {
        'Domingo': 1,
        'Segunda-feira' : 2,
        'Terça-feira' : 3,
        'Quarta-feira' : 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado' : 7
    }

    return dias_semana.get(dia)


In [28]:
indice_semana('Sábado')

7

In [29]:
df_publicacao_video  = spark.sql(' SELECT SEMANA_TRADUZIDA,  sum(TOTAL_VIDEOS) '
          ' FROM PUBLICACAO_VIDEO'
          ' GROUP BY SEMANA_TRADUZIDA ').toPandas()
df_publicacao_video['INDICE_SEMANA'] = df_publicacao_video['SEMANA_TRADUZIDA'].apply(indice_semana)
df_publicacao_video.sort_values(by='INDICE_SEMANA', inplace=True)
df_publicacao_video.head(7)


,SEMANA_TRADUZIDA,sum(TOTAL_VIDEOS),INDICE_SEMANA
5,Domingo,25,1
0,Segunda-feira,40,2
2,Terça-feira,39,3
1,Quarta-feira,37,4
6,Quinta-feira,39,5
4,Sexta-feira,34,6
3,Sábado,21,7


In [30]:
fig = px.bar(df_publicacao_video, x='SEMANA_TRADUZIDA',y='sum(TOTAL_VIDEOS)', text_auto=True)
fig.show()

# Total de Tags X Vísualizações 

In [42]:
df_tags_visualizacoes = spark.sql('SELECT TOTAL_TAGS, TOTAL_VISUALIZACOES FROM METRICAS_VIDEO where ID_CANAL in ("UCrOH1V-FyMunBIMrKL0y0xQ") ').toPandas()
df_tags_visualizacoes.head()
fig = px.scatter(df_tags_visualizacoes, x='TOTAL_TAGS', y='TOTAL_VISUALIZACOES')
fig.show()

# Tamanho dos títulos do Vídeos X Vísualizações

In [46]:
df_tamanho_titulo_vi = spark.sql('SELECT TOTAL_CARACTERE_VIDEO, TOTAL_VISUALIZACOES FROM METRICAS_VIDEO').toPandas()
fig = px.scatter(df_tamanho_titulo_vi, x='TOTAL_CARACTERE_VIDEO', y='TOTAL_VISUALIZACOES')
fig.show()


# Total Víews DIA por Vídeo

In [76]:
df_views_video = spark.sql(' SELECT TITULO_VIDEO, TOTAL_VISUALIZACOES_DIA '  
          ' FROM METRICAS_VIDEO ' 
          ' WHERE ID_CANAL = "UCrOH1V-FyMunBIMrKL0y0xQ" ' 
          ' AND data_extracao = "2023-10-27" '
          ' AND TOTAL_VISUALIZACOES > 0 '
          ' ORDER BY 2 ').toPandas()
df_views_video.head() 
fig = px.bar(df_views_video, x='TOTAL_VISUALIZACOES_DIA', y='TITULO_VIDEO', orientation='h', text_auto=True)
fig.show()

# A duração do vídeo é importante para visualizações e interação (curtidas/comentários)?

# Desepenho dos vídeos (like, comentários, vísualizações) (Faça comparação)

# Desepenho dos Canais (like, comentários, vísualizações) Faça Comparação


# Total De Vídeo por duração ex: entre dois e três minutos, eu tenho quantos vídeos

- Métricas especificas TRENDS